In [6]:
#CHECK CONNECTION

import mysql.connector

# Establishing a connection to the database
cnx = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='myshop')

# Closing the connection
cnx.close()

In [7]:
# Establishing a connection to the database
cnx = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='myshop')

# Creating a cursor object
cursor = cnx.cursor()

# Executing a SELECT statement
# Inserting a new row
query = "INSERT INTO supplier (supplier_name, phone, email) VALUES ('Coffee maker','28594679','coffmaker@make.com')"
cursor.execute(query)

# Committing the transaction
cnx.commit()

# Closing the cursor and connection
cursor.close()
cnx.close()


In [8]:
# Establishing a connection to the database
cnx = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='myshop')

# Creating a cursor object
cursor = cnx.cursor()

# Executing a SELECT statement
# Inserting a new row
query = "select * from supplier"
cursor.execute(query)

# Fetching the results
for row in cursor.fetchall():
    print(row)

# Closing the cursor and connection
cursor.close()
cnx.close()

(1, 'Coffee maker', 28594679, 'coffmaker@make.com')


In [14]:
# Establishing a connection to the database
cnx = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='myshop')

# Creating a cursor object
cursor = cnx.cursor()

name = input("Enter the name of the supplier:\n")
phone = input("Enter the phone of the supplier:\n")
email = input("Enter the email of the supplier:\n")

# Executing a SELECT statement
# Inserting a new row
query = "INSERT INTO supplier (supplier_name, phone, email) VALUES (%s,%s,%s)"
data = (name,phone,email)
cursor.execute(query,data)

# Committing the transaction
cnx.commit()

# Closing the cursor and connection
cursor.close()
cnx.close()

Enter the name of the supplier:
Choco
Enter the phone of the supplier:
28594769
Enter the email of the supplier:
choco@gmail.com


In [20]:
# Establishing a connection to the database
cnx = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='myshop')

# Creating a cursor object
cursor = cnx.cursor()

id = int(input("Enter the ID of the supplier:\n"))
# Creating a cursor object
cursor = cnx.cursor()

# Executing a SELECT statement
# Inserting a new row
query = "select * from supplier where id = %s"
data = (id,)#tuple of one element
cursor.execute(query,data)

# Fetching the results
for row in cursor.fetchall():
    print(row)

# Closing the cursor and connection
cursor.close()
cnx.close()

Enter the ID of the supplier:
3


In [40]:
import pandas as pd
import pymysql

from sqlalchemy import create_engine
from sqlalchemy import text

# # Establishing a connection to the database
# cnx = mysql.connector.connect(user='root',
#                               password='',
#                               host='localhost',
#                               database='myshop')


engine = create_engine("mysql+pymysql://root:@localhost/myshop")
# query = "SELECT * FROM supplier"


with engine.begin() as con:
    # suppliers_df = pd.DataFrame()
    query = text("SELECT * FROM supplier")
    suppliers_df = pd.read_sql(query,con)
    # cnx.close()


In [41]:
suppliers_df

,id,supplier_name,phone,email
0,1,Coffee maker,28594679,coffmaker@make.com
1,2,Fruit paradise,28564968,frtparad@gmail.com
2,4,Choco,28594769,choco@gmail.com


In [48]:
import pandas as pd
import pymysql

from sqlalchemy import create_engine
from sqlalchemy import text

engine = create_engine("mysql+pymysql://root:@localhost/myshop")

with engine.begin() as con:
    query = text("SELECT * FROM supplier")
    suppliers_df = pd.read_sql(query,con)
    suppliers_df.to_csv("Suppliers.csv",index=False,encoding="utf8",sep=";")
    suppliers_df.to_excel("Suppliers.xlsx",index=False)


In [55]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/myshop")

df = pd.read_excel(r"C:\Users\arturs.olekss\OneDrive - Grupo VASS\PythonCourse\Practical\SQL\Suppliers.xlsx")
df.to_sql(name="supplier",con=engine,if_exists="append",index=False)


3

,supplier_name,phone,email
0,Juice maker,2897679,juic.make@gmail.com
1,Vegy garden,21568969,veggard@yahoo.com
2,Tea garden,25498016,tg@gmail.com


In [71]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/myshop")

categories = pd.read_excel(r"C:\Users\arturs.olekss\OneDrive - Grupo VASS\PythonCourse\Practical\SQL\Categories.xlsx",
                          converters={"name":str,"external_id":str},names=("name","external_id"))
categories.to_sql(name="categories",con=engine,if_exists="append",index=False)

3

,name,external_id
0,Fruits,0001
1,Vegetables,0002
2,Drinks,0003


In [88]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/myshop")

products = pd.read_excel(r"C:\Users\arturs.olekss\OneDrive - Grupo VASS\PythonCourse\Practical\SQL\Products.xlsx",
                          converters={"price":float,"category":str})
with engine.begin() as con:
    query = text("SELECT * FROM categories")
    categories_db = pd.read_sql(query,con)

products = products.merge(categories_db,left_on="category",right_on="external_id")
products = products.drop(columns=["name","external_id","category"])

col_ren = {"id":"category"}
products.columns = [col_ren.get(x,x) for x in products.columns]#rename
products.to_sql(name="products",con=engine,if_exists="append",index=False)

2

In [101]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/myshop")
with engine.begin() as con:
    query = text("SELECT "+
                 " a.id, a.title, a.description, a.price, a.warranty, b.name, c.supplier_name, c.phone AS 'supplier_phone'," + 
                 " c.email AS 'supplier_email'"+
                 " FROM products AS a INNER JOIN categories AS b ON a.category = b.id INNER JOIN supplier AS c"+
                " ON c.id = a.supplier_id")
    products_db = pd.read_sql(query,con)

In [104]:
products_db.to_excel("Products_database.xlsx")

In [110]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/myshop")

statuses = pd.read_excel(r"C:\Users\arturs.olekss\OneDrive - Grupo VASS\PythonCourse\Practical\SQL\Statuses.xlsx",
                          converters={"ext_id":str})
statuses.to_sql(name="status",con=engine,if_exists="append",index=False)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '0001' for key 'ext_id'")
[SQL: INSERT INTO status (text, ext_id) VALUES (%(text)s, %(ext_id)s)]
[parameters: [{'text': 'Open', 'ext_id': '0001'}, {'text': 'In Progress', 'ext_id': '0002'}, {'text': 'Delivered', 'ext_id': '0003'}]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [112]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/myshop")

customers = pd.read_excel(r"C:\Users\arturs.olekss\OneDrive - Grupo VASS\PythonCourse\Practical\SQL\Customers.xlsx",
                          converters={"ext_id":str})
customers.to_sql(name="customer",con=engine,if_exists="append",index=False)

2

In [190]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/myshop")

orders = pd.read_excel(r"C:\Users\arturs.olekss\OneDrive - Grupo VASS\PythonCourse\Practical\SQL\Orders.xlsx")
orders_products = orders[["products","ext_id"]]
ext_ids = orders["ext_id"]
orders = orders.drop("products",axis=1)

with engine.begin() as con:
    query = text("SELECT id,ext_id FROM customer WHERE ext_id IN " + str(tuple(orders["customer_id"])))
    customers_db = pd.read_sql(query,con)


orders = orders.merge(customers_db,left_on="customer_id",right_on="ext_id")
orders = orders.drop(["customer_id","ext_id_y"],axis=1)


col_ren = {"id":"customer_id"}
orders.columns = [col_ren.get(x,x) for x in orders.columns]#rename

col_ren = {"ext_id_x":"ext_id"}
orders.columns = [col_ren.get(x,x) for x in orders.columns]#rename
    
orders.to_sql(name="orders",con=engine,if_exists="append",index=False)

orders_products_formatted = pd.DataFrame(columns=["order_id","product_id","product_amount"])
for row_indx,row in orders_products.iterrows():
    products = row["products"].split(";")
    for product in products:
        product_details = product.split(":")
        orders_products_formatted.loc[len(orders_products_formatted)] = [row["ext_id"],product_details[0],product_details[1]]

with engine.begin() as con:
    query = text("SELECT id,ext_id FROM orders WHERE ext_id IN " + str(tuple(orders["ext_id"])))
    order_ids = pd.read_sql(query,con)


orders_products_formatted  = orders_products_formatted .merge(order_ids,left_on="order_id",right_on="ext_id")
orders_products_formatted = orders_products_formatted.drop(["order_id","ext_id"],axis=1)
col_ren = {"id":"order_id"}
orders_products_formatted.columns = [col_ren.get(x,x) for x in orders_products_formatted.columns]#rename
orders_products_formatted.to_sql(name="order_products",con=engine,if_exists="append",index=False)

3